In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import selfies as sf
import os
from rdkit import Chem
from rdkit.Chem import MolFromSmiles
from rdkit.Chem import MolToSmiles
from rdkit.Chem import DataStructs
from rdkit.Chem import AllChem

In [2]:
# Define the CHEMENV class (same as in your training script)
class CHEMENV:
    def __init__(self, max_actions=150):
        self.tokens = ['[#Branch1]', '[#Branch2]', '[=Branch1]', '[=Branch2]', '[=Ring1]', '[=Ring2]', '[Branch1]', '[Branch2]', '[Ring1]', '[Ring2]']
        self.tokens += ['[P]', '[O]', '[N]', '[F]', '[Cl]', '[C]', '[Br]', '[#C]', '[S]']
        self.tokens += ['[=C]', '[=N]', '[=O]', '[=P]', '[=S]', '[#N]'] #, '[=S+1]', '[=PH1]']
        self.action_space = ['<', '>'] + self.tokens
        self.atoi = {a: i for i, a in enumerate(self.action_space)}
        self.current_state = [0]
        self.max_actions = max_actions
        self.action_count = 0

    def get_num_action(self):
        return len(self.action_space)

    def reset(self):
        self.current_state = [0]
        self.action_count = 0
        return self.current_state

    def to_index_list(self, s):
        return [self.atoi[a] for a in s]

    def to_selfies(self, s):
        selfi = "".join([self.action_space[i] for i in s])
        selfi = selfi.replace(self.action_space[1], "").replace(self.action_space[0], "")
        return selfi

    def calc_reward(self, selfies1):
        try:
            smiles1 = sf.decoder(selfies1)
        except sf.DecoderError as e:
            return False
        try:
            mol1 = MolFromSmiles(smiles1)
            if mol1 is not None:
                ms = [Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles('COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C)OCCOC')]
                fpgen = AllChem.GetRDKitFPGenerator()
                fps = [fpgen.GetFingerprint(x) for x in ms]
                tanimoto = DataStructs.TanimotoSimilarity(fps[0],fps[1])
                return tanimoto * 3
            else:
                return 0
        except Exception as e:
            print(f"Error in calc_reward: {e}")  # Debug print
            return 0

    def step(self, a):
        self.current_state = self.current_state + [a]
        self.action_count += 1
        selfies_string = self.to_selfies(self.current_state)
        r = self.calc_reward(selfies_string)
        terminated = (a == 1) and (len(self.current_state) > 5)  # 최소 5자 이상 생성하는 조건 추가
        terminated = terminated or self.action_count >= self.max_actions
        return self.current_state, r, terminated, False, None

    def close(self):
        pass

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Actor 모델 정의 (same as in your training script)
class Actor(nn.Module):
    def __init__(self, num_action, embedding_dim, hidden_size):
        super(Actor, self).__init__()
        self.emb = nn.Embedding(num_action, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_size, num_layers=4, batch_first=True, dropout=0.2)  # 여러 레이어 추가
        self.fc = nn.Linear(hidden_size, num_action)

    def forward(self, state):
        ## type(state): tensor
        x = self.emb(state.unsqueeze(0))  # 배치 차원 추가
        o, h = self.rnn(x)  # o.shape: (batch, seqlen, hidden_size), h.shape: (num_layers, batch, hidden_size)
        o_last = self.fc(o[:, -1, :])  # 배치의 마지막 시퀀스 아이템
        probabilities = torch.softmax(o_last, dim=-1)
        return probabilities

# Critic 모델 정의 (same as in your training script)
class Critic(nn.Module):
    def __init__(self, num_action, embedding_dim, hidden_size):
        super(Critic, self).__init__()
        self.emb = nn.Embedding(num_action, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_size, num_layers=4, batch_first=True, dropout=0.2)  # 여러 레이어 추가
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, state):
        ## type(state): tensor
        x = self.emb(state.unsqueeze(0))  # 배치 차원 추가
        o, h = self.rnn(x)  # o.shape: (batch, seqlen, hidden_size), h.shape: (num_layers, batch, hidden_size)
        o_last = self.fc(o[:, -1, :])  # 배치의 마지막 시퀀스 아이템
        return o_last

In [5]:
# Sampling function
def sample_from_model(actor, env, num_samples=10):
    actor.eval()
    samples = []
    for _ in range(num_samples):
        state = env.reset()
        done = False
        while not done:
            state_tensor = torch.IntTensor(state).to(device)
            probs = actor(state_tensor)
            dist = torch.distributions.Categorical(probs)
            action = dist.sample()

            next_state, reward, terminated, truncated, _ = env.step(action.item())
            done = terminated or truncated

            state = next_state

        selfies_string = env.to_selfies(state)
        try:
            smiles = sf.decoder(selfies_string)
            samples.append(smiles)
        except sf.DecoderError:
            samples.append(None)
    
    return samples

In [6]:
# Load checkpoint
checkpoint_path = 'ppo_checkpoints/checkpoint_3000.pth'
checkpoint = torch.load(checkpoint_path, map_location=device)
# Initialize environment and actor model
env = CHEMENV(max_actions=150)
actor = Actor(env.get_num_action(), 16, 32).to(device)
actor.load_state_dict(checkpoint['actor_state_dict'])

<All keys matched successfully>

In [7]:
# Reference molecule for Tanimoto similarity calculation
reference_smiles = 'COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C)OCCOC'
reference_mol = Chem.MolFromSmiles(reference_smiles)
fpgen = AllChem.GetRDKitFPGenerator()
ref_fp = fpgen.GetFingerprint(reference_mol)

In [8]:
num_samples = 500  # Number of samples to generate
samples = sample_from_model(actor, env, num_samples=num_samples)

In [9]:
tanimoto_scores = []

for i, smi in enumerate(samples):
    if smi is not None:
        sample_mol = Chem.MolFromSmiles(smi)
        if sample_mol is not None:
            sample_fp = fpgen.GetFingerprint(sample_mol)
            tanimoto = DataStructs.TanimotoSimilarity(ref_fp, sample_fp)
            tanimoto_scores.append(tanimoto)
            print(f"Sample {i + 1}: {smi}, Tanimoto score: {tanimoto}")
        else:
            tanimoto_scores.append(None)
            print(f"Sample {i + 1}: {smi}, Tanimoto score: None (Invalid molecule)")
    else:
        tanimoto_scores.append(None)
        print(f"Sample {i + 1}: {smi}, Tanimoto score: None (Decoding error)")

# Calculate max, min, and average Tanimoto scores
valid_scores = [score for score in tanimoto_scores if score is not None]
max_tanimoto = max(valid_scores) if valid_scores else None
min_tanimoto = min(valid_scores) if valid_scores else None
avg_tanimoto = sum(valid_scores) / len(valid_scores) if valid_scores else None

print()
print(f"Max Tanimoto score: {max_tanimoto:.3f}")
print(f"Min Tanimoto score: {min_tanimoto:.3f}")
print(f"Average Tanimoto score: {avg_tanimoto:.3f}")


# Print samples sorted by Tanimoto score in descending order
sorted_samples = sorted(zip(samples, tanimoto_scores), key=lambda x: (x[1] is not None, x[1]), reverse=True)
print("\nSamples sorted by Tanimoto score (in descending order):")
for i, (smi, score) in enumerate(sorted_samples):
    print(f"Sample {i + 1}: {smi}, Tanimoto score: {score}")

Sample 1: NNNNNNN=NNNN=NN=NN=NNNNNNN=NNNN=NNN=NON=NNNN=NN=NN=NNNN=NN=NN=NN(N)N=NN=NNN=NNNNN=NN=NN=NNNN=NNNN=NNN(N)N=NN, Tanimoto score: 0.06222707423580786
Sample 2: NNNNNNNNNNNNNNNNNNNN=NNN=NNN=NNN=NNNN=NN=NN=NN=NN=NN=NN=NN=NN(N)N=NN=NN=NN=NN=NN=NN=NN=NN=NN=NN=NN=NN(N)NN=NN=NN=NN=NN(N)N=NN=NNN=NN=NN=NNN=NN(N)N=NN=NN, Tanimoto score: 0.043721973094170405
Sample 3: ONNNNNNNNNNNNN=NNN=NNNN=NN=NN=NN=NNN(N)NNN=NNN=NNN=NNN(N=NN=NN)N=NN=NN=NN=NN=NN=NN=NNNN=NN=NN(N)NNN=NN=NN=NNN=NN=NOO, Tanimoto score: 0.06759656652360516
Sample 4: NNNNNNNNNNNNNNNN=NN=NNNNNN=NN=NN=NN=NN=NN=NN(N)N=NNN=NNN=NN=NN(N=NONN)N(O)NN=NN=NN=NN=NN=NN(N)N=NNN=NN=NN(N(N)N=N)N=NN=NN=NNN=NN=N, Tanimoto score: 0.09566968781470292
Sample 5: NNNNNNNN=O, Tanimoto score: 0.015439429928741092
Sample 6: NNNNNNNN=NNNNN=NN=NN=NN=NN=NN=NN=NN=NNN=NN=NN=NN=NN=NN=NN=NN=NNN=NNN(N)N=NN=NNN(N)N(N)N=NN=NN(N)NNNNNN=NN=NNN=NN=NN=NN(N)N=NN=NN=NN=NN, Tanimoto score: 0.054764512595837894
Sample 7: NNNNNNNNNNNNNNNN=NNNNN=NN=NNN=NNN=NN=NN=NNN(N)N=N